In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Intenciones:
1. Cargar los datos de entrada
1. Acondicionar los datos. Separar datos de entrenamiento de validación
1. Explorar distintos features.
1. Armar un DataFrame de entrenamiento y otro de validación (poner nombre acá)
1. Explorar distintos features
1. Elegir varios features a mano
1. Armar algoritmo para elegir features (random forests?) => ver esto
1. Entrenar diferentes modelos y de diferentes formas.
1. Iterar sobre los puntos anteriores
1. Registrar los cambios y evoluciones
1. Sacar conclusiones
## Modificar esta lista acorde a lo que se vaya haciendo.
### Nota: La intención es dejar comentados varios campos a modo de debug que generan salidas muy verbosas pero pueden resultar útiles en alún momento (ejemplo, .head() )

## 1) Cargar los datos de entrada

In [3]:
#_PATH_DATASET = '../input/all/' # No borrar esto, por compatibilidad para correrlo con datasets en otros directorios
_PATH_DATASET = '/home/diego/Cursos/FIUBA/git/fiuba-trocafone-tp2-final-set/' # No borrar esto, por compatibilidad para correrlo con datasets en otros directorios

In [4]:
input_data_raw = pd.read_csv(_PATH_DATASET + 'events_up_to_01062018.csv')
input_data_raw.head()

/home/diego/.local/anaconda3/envs/tfdeeplearning/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
pd.DataFrame(input_data_raw.columns)

,0
0,timestamp
1,event
2,person
3,url
4,sku
5,model
6,condition
7,storage
8,color
9,skus


In [5]:
# Se cargan los labels
knownLabels = pd.read_csv(_PATH_DATASET + 'labels_training_set.csv')
knownLabels['resultKnown'] = 1
knownLabels.head()

,person,label,resultKnown
0,0566e9c1,0,1
1,6ec7ee77,0,1
2,abe7a2fb,0,1
3,34728364,0,1
4,87ed62de,0,1


In [6]:
sum(knownLabels.label)

980

# 2) Acondicionar los datos (previo al armado del DataFrame final)

In [7]:
# Acondicionar los datos antes de convertirlos
input_data_raw['timestamp'] = pd.to_datetime(input_data_raw['timestamp']) # Convierto a timestamp

In [8]:
input_data_raw['timestamp'].dt.week.min()

1

In [9]:
print('Total de datos: ' + str(input_data_raw.shape))
print('Total de labels: ' + str(knownLabels.shape))

Total de datos: (2341681, 23)
Total de labels: (19414, 3)


In [10]:
# Hubo un cambio grande, a todos los trato por igual y luego en algún momento los separo por label 0 o 1 o
# en testing, -1
input_train = input_data_raw# [input_data_raw.person.isin(knownLabels.person)] # Datos etiquetados
input_to_be_validated = input_data_raw[(~input_data_raw.person.isin(knownLabels.person))] # Datos para predecir

In [11]:
# Para simplificar el preprocesamiento, hago un array único en donde las labels desconocidas son las -1
unknownLabels = pd.DataFrame(input_to_be_validated['person'])

In [12]:
unknownLabels['label'] = 0

In [13]:
unknownLabels = unknownLabels.groupby('person')

In [14]:
unknownLabels = unknownLabels.count()
unknownLabels['resultKnown'] = 0
unknownLabels['label'].values[:] = 0
#unknownLabels.label = 0 # Si seteo a 0, no queda como un DataFrame

In [15]:
unknownLabels.head()

,label,resultKnown
person,,
00091926,0,0
00091a7a,0,0
000ba417,0,0
000e4d9e,0,0
000e619d,0,0


In [16]:
print('Total de labels desconocidas: ' + str(unknownLabels.shape))

Total de labels desconocidas: (19415, 2)


In [17]:
knownLabels = knownLabels.set_index('person')
#unknownLabels = unknownLabels.set_index('person')

joinedLabels = [knownLabels, unknownLabels]

In [18]:
joinedLabels = pd.concat(joinedLabels)
joinedLabels.head()

,label,resultKnown
person,,
0566e9c1,0,1
6ec7ee77,0,1
abe7a2fb,0,1
34728364,0,1
87ed62de,0,1


In [19]:
print('Total de labels: ' + str(joinedLabels.shape))

Total de labels: (38829, 2)


In [20]:
print('Total de datos con label conocidos: ' + str(input_train.shape[0])) # Datos 
print('Total de datos a predecir ' + str(input_to_be_validated.shape[0]))
print('Total de datos (debe coincidir) ' + str(input_train.shape[0] + input_to_be_validated.shape[0]))

Total de datos con label conocidos: 2341681
Total de datos a predecir 1169795
Total de datos (debe coincidir) 3511476


# Acondiciono los datos por persona (ahora tengo una entrada por evento y no por persona)

In [21]:
# input_train.describe()
# input_train.info()

### Nota: SKU: Stock-keeping unit o SKU, sin traducción literal al español podría denominarse «código de artículo» o «número de referencia» o incluso un código de barras. Asignado a un elemento para poder identificarlo en el inventario físico y/o financiero.

# ACLARACIÓN:
## Borrar lo que no necesito de acá luego, en esto hay mucho borrador, lo importante es llegar al DataFrame de features...

In [22]:
input_train = input_train.sort_values(['person']) # ordeno por persona, no es necesario si voy a stackear

In [23]:
# Me aseguro de que todos los eventos tienen el campo persona de 8 caracteres
# input_train['person'].apply(lambda x: len(x)).value_counts() # => OK

In [24]:
runTests = False

In [25]:
# Desprolijo, después aprovechar esto.
if runTests == True:
    in2 = input_train.set_index('person')
    in2.head()

In [26]:
if runTests == True:
    # input_train.sort_values(['person', 'timestamp'])
    # input_train.groupby('person').count()
    input_train_features = input_train.groupby('person').count() # Este no será el dato final

In [27]:
#input_train_features.columns

In [28]:
if runTests == True:
    total = input_train_features.join(knownLabels, on = 'person', how = 'left')

In [29]:
# total.shape
# total.label.head()

In [30]:
if runTests == True:
    x_data = total.drop('label', axis = 1)
    y_labels = total.label
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(x_data,y_labels,test_size=0.3,random_state=101)
    x_data.columns

## Esto también es una prueba

In [31]:
if runTests == True:
    import tensorflow as tf

In [32]:
if runTests == True:
    #feat_cols = tf.convert_to_tensor(x_data) # No anda, da errores al momento de entrenar (model.train)
    ts = tf.feature_column.numeric_column("timestamp")
    event = tf.feature_column.numeric_column("event")
    url = tf.feature_column.numeric_column("url")
    model = tf.feature_column.numeric_column("model")
    condition = tf.feature_column.numeric_column("condition")
    storage = tf.feature_column.numeric_column("storage")
    color = tf.feature_column.numeric_column("color")
    search_term = tf.feature_column.numeric_column("search_term")

    feat_cols = [ts, event, url]

In [33]:
if runTests == True:
    type(feat_cols)

In [34]:
if runTests == True:
    input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

In [35]:
if runTests == True:
    model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

In [36]:
if runTests == True:
    model.train(input_fn=input_func,steps=5000)

In [37]:
if runTests == True:
    pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [38]:
if runTests == True:
    predictions = list(model.predict(input_fn=pred_fn))

In [39]:
if runTests == True:
    final_preds = []
    for pred in predictions:
        final_preds.append(pred['class_ids'][0])

In [40]:
if runTests == True:
    from sklearn.metrics import classification_report

In [41]:
if runTests == True:
    print(classification_report(y_test,final_preds))

In [42]:
if runTests == True:
    input_train[input_train['person'] == '0008ed71'].sort_values(['timestamp'])

In [43]:
if runTests == True:
    input_train.loc[1505383]['skus']

In [44]:
if runTests == True:
    input_train[input_train['skus'].isnull()]

In [45]:
if runTests == True:
    input_train.loc[2122051].skus

## FIN DE PRUEBAS. Sigo con el espacio de features nuevamente

# 3) Explorar distintos features

## Creo el nuevo vector de features.
### El mismo consiste en un DataFrame del que se evaluarán luego cuáles usar

In [46]:
input_train.person.head()

1507286    0008ed71
2336761    0008ed71
2122051    0008ed71
1505383    0008ed71
2336760    0008ed71
Name: person, dtype: object

In [47]:
joinedLabels.head()

,label,resultKnown
person,,
0566e9c1,0,1
6ec7ee77,0,1
abe7a2fb,0,1
34728364,0,1
87ed62de,0,1


In [48]:
new_vector_features = pd.DataFrame(joinedLabels.index) # Creo un vector con una entrada por persona
#new_vector_features['_'] = 0
new_vector_features = new_vector_features.set_index('person')
# new_vector_features.head()

In [49]:
# Agrego los labels
new_vector_features = new_vector_features.join(joinedLabels, on = 'person', how = 'left')

In [50]:
new_vector_features[new_vector_features['resultKnown']!=1].shape

(19415, 2)

## Estudio los features útiles. Crear lista de ideas

In [51]:
input_train.columns.values # Lista de features crudos disponibles. Desde ellos debo realizar los diferentes

array(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version'], dtype=object)

## Demo de groupby, por alguna duda en particular

In [52]:
df = pd.DataFrame({'A': 'a a b a'.split(), 'B': [1,2,3,1.5], 'C': [4,6, 5, 100]})
g = df.groupby('A')
g.head()

,A,B,C
0,a,1.0,4
1,a,2.0,6
2,b,3.0,5
3,a,1.5,100


In [53]:
g.apply(lambda x: x.sum())

,A,B,C
A,,,
a,aaa,4.5,110
b,b,3.0,5


## Features de timestamp

In [54]:
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.apply.html
# Ejecuto una función para cada grupo:
grouped_input_train = input_train.groupby('person')
grouped_input_train.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
1507286,2018-05-17 12:27:47,checkout,0008ed71,NaN,3372.0,Samsung Galaxy S6 Flat,Muito Bom,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2336761,2018-05-17 16:21:54,visited site,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Referral,Returning,Unknown,Unknown,Brazil,Computer,1920x1080,Windows 10,Chrome 66.0
2122051,2018-05-17 16:22:06,generic listing,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,"6594,6651,6664,7253,2820,6706,6721,12606,480,1...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1505383,2018-05-17 16:28:37,checkout,0008ed71,NaN,7505.0,LG G4 H818P,Bom,32GB,Preto,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2336760,2018-05-17 13:44:59,visited site,0008ed71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Referral,New,Unknown,Unknown,Brazil,Computer,1920x1080,Windows 10,Chrome 66.0
227610,2018-05-23 01:09:18,ad campaign hit,00091926,/comprar/iphone/6s,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244622,2018-05-17 00:56:34,viewed product,00091926,NaN,10309.0,iPhone 7 Plus,Muito Bom,256GB,Ouro Rosa,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175432,2018-05-31 02:33:58,viewed product,00091926,NaN,6846.0,iPhone 6S,Muito Bom,16GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244443,2018-05-17 01:13:22,viewed product,00091926,NaN,1389.0,iPhone 6 Plus,Excelente,16GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252233,2018-05-07 00:37:12,viewed product,00091926,NaN,10910.0,Samsung Galaxy A7 2017,Bom,32GB,Rosa,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
debugging = True

In [56]:
if debugging == True:
    print(input_train['model'].unique()[0:10])

['Samsung Galaxy S6 Flat' nan 'LG G4 H818P' 'iPhone SE' 'iPhone 7 Plus'
 'iPhone 6S' 'iPhone 6 Plus' 'Samsung Galaxy A7 2017' 'iPhone 6S Plus'
 'iPhone 7']


In [57]:
if debugging == True:
    print(input_train['url'].unique())

[nan '/comprar/iphone/6s' '/comprar/samsung/galaxy-s6-edge-plus'
 '/comprar/iphone' '/comprar/iphone/7-plus' '/comprar/iphone/6-plus'
 '/comprar/iphone/iphone-6s-plus' '/comprar/samsung/galaxy-s7-edge'
 '/comprar/samsung/galaxy-s6-flat' '/comprar/samsung'
 '/comprar/samsung/galaxy-s8-plus' '/comprar/samsung/galaxy-s8'
 '/comprar/iphone/' '/comprar/samsung/a3-duos' '/'
 '/comprar/samsung/galaxy-s5' '/comprar/lg/g3-d855'
 '/comprar/iphone/iphone-se' '/comprar/samsung/samsung-gran-prime-duos-tv'
 '/comprar/samsung/galaxy-s7' '/comprar/samsung/galaxy-win-duos'
 '/comprar/motorola/moto-g-2a-geracao' '/comprar/iphone/iphone-5c'
 '/comprar/samsung/galaxy-j1-mini' '/comprar/samsung/galaxy-s4-mini-duos'
 '/comprar/motorola' '/comprar/samsung/galaxy-s5-duos' '/comprar/motorola/'
 '/comprar/samsung/galaxy-j7-prime' '/comprar/iphone/6'
 '/comprar/iphone/iphone-5s' '/comprar/motorola/moto-x-play-4g-dual'
 '/vender/' '/comprar/samsung/a5'
 '/comprar/samsung/galaxy-core-plus-duos-tv'
 '/comprar/samsu

In [58]:
if debugging == True:
    print(input_train['storage'].unique())

['32GB' nan '64GB' '256GB' '16GB' '128GB' '8GB' '4GB' '512MB']


In [59]:
if debugging == True:
    print(input_train['color'].unique()[0:50])

['Dourado' nan 'Preto' 'Cinza espacial' 'Ouro Rosa' 'Rosa' 'Preto Matte'
 'Prateado' 'Vermelho' 'Preto Brilhante' 'Azul' 'Black Piano' 'Branco'
 'Bambu' 'Ametista' 'Ouro' 'Azul Escuro' 'Verde' 'Platinum' 'Prata'
 'Verde Água' 'Couro Navy' 'Cinza' 'Rose' 'Branco Vermelho' 'Iuna'
 'Preto Azul' 'Branco Azul' 'Preto Pink' 'Preto Azul Navy' 'Rouge'
 'Couro Vintage' 'Branco Bambu' 'Cabernet' 'Branco Azul Navy' 'Preto Verde'
 'Branco Dourado' 'Coral' 'Titânio' 'Roxo' 'Cromo' 'Olympic Edition'
 'Azul Safira' 'Preto Vermelho' 'Branco Pink' 'Indigo' 'Azul Topázio'
 'Amarelo' 'Branco Framboesa' 'Turquesa']


In [60]:
#input_train['skus'].unique()
#input_train['sku'].unique()

In [61]:
if debugging == True:
    print(input_train['search_term'].unique())

[nan 'Iphone 7s' 'Galaxy s8' ..., 'iPhone 6 Rose' '5S Mini' 'J5 Golden']


In [62]:
if debugging == True:
    print(input_train['staticpage'].unique())

[nan 'Conditions' 'CustomerService' 'Quiosks' 'FaqEcommerce' 'galaxy-s8'
 'how-to-buy' 'TermsAndConditionsReturnEcommerce' 'AboutUs'
 'TermsAndConditionsEcommerce' 'trust-trocafone' 'club-trocafone'
 'how-to-sell' 'black_friday' 'PrivacyEcommerce']


In [63]:
if debugging == True:
    print(input_train['campaign_source'].unique())

[nan 'google' 'rtbhouse' 'criteo' 'afilio' 'zanox' 'rakuten' 'voxus'
 'buscape' 'indexa' 'emblue' 'Facebook' 'FacebookAds' 'FacebookSocial'
 'bing' 'manifest' 'onsite' 'datacrush' 'yotpo' 'blog' 'mercadopago'
 'afiliado' 'MARKETING SOCIAL' 'gizmodo']


In [64]:
if debugging == True:
    print(input_train['search_engine'].unique())

[nan 'Google' 'Yahoo' 'Bing' 'Ask']


In [65]:
if debugging == True:
    print(input_train['channel'].unique())

[nan 'Referral' 'Direct' 'Paid' 'Organic' 'Social' 'Email' 'Unknown']


In [66]:
if debugging == True:
    print(input_train['new_vs_returning'].unique())

[nan 'Returning' 'New']


In [67]:
if debugging == True:
    print(input_train['city'].unique())

[nan 'Unknown' 'Carlos Barbosa' ..., 'Marcos Parente' 'Cha Grande'
 'Barra dos Coqueiros']


In [68]:
if debugging == True:
    print(input_train['region'].unique()[0:50]) # Sólo muestro un par de elementos

[nan 'Unknown' 'Rio Grande do Sul' 'Minas Gerais' 'Sao Paulo'
 'Rio de Janeiro' 'Mato Grosso' 'Mato Grosso do Sul' 'Santa Catarina'
 'Parana' 'Tocantins' 'Ceara' 'Federal District' 'Bahia' 'Pernambuco'
 'Amazonas' 'Goias' 'Rio Grande do Norte' 'Maranhao' 'Alagoas' 'Acre'
 'Paraíba' 'Para' 'Espirito Santo' 'California' 'Departamento de La Paz'
 'Arkansas' 'Rondonia' 'Piaui' 'Sergipe' 'New Jersey' 'Texas' 'Illinois'
 'Asuncion' 'Departamento de Montevideo' 'Aveiro' 'New York'
 'Cidade de Maputo' 'Florida' 'Roraima' 'Arizona' 'Amapa'
 'Buenos Aires F.D.' 'Morelos' 'Ohio' "Tul'skaya Oblast'" 'North Holland'
 'Neuquen' 'Dhaka' 'Tinh Thua Thien-Hue']


In [69]:
if debugging == True:
    print(input_train['country'].unique()[0:500])

[nan 'Brazil' 'Unknown' 'United States' 'Bolivia' 'Paraguay' 'Uruguay'
 'Portugal' 'Mozambique' 'Argentina' 'Mexico' 'Russia' 'Netherlands'
 'Bangladesh' 'Vietnam' 'Canada' 'Republic of Korea' 'United Kingdom'
 'Angola' 'Morocco' 'Burundi' 'Cape Verde' 'Belgium' 'Georgia' 'Guadeloupe'
 'France' 'South Africa' 'Sweden' 'Israel' 'Germany' 'Singapore' 'Bulgaria'
 'Colombia' 'Austria' 'Cameroon' 'Spain' 'Japan' 'Peru' 'Costa Rica'
 'Italy' 'India' 'Dominican Republic' 'Ireland' 'Slovak Republic' 'Jamaica'
 'Pakistan' 'Guinea-Bissau' 'Ukraine' 'Philippines' 'São Tomé and Príncipe'
 'Algeria' 'Romania']


In [70]:
if debugging == True:
    print(input_train['device_type'].unique())

[nan 'Computer' 'Smartphone' 'Tablet' 'Unknown']


In [71]:
if debugging == True:
    print(input_train['screen_resolution'].unique()[0:100])

[nan '1920x1080' '1024x768' '360x640' '1536x864' '1366x768' '320x534'
 '412x846' '1360x768' '1242x698' '1138x640' '1280x720' '375x667' '320x522'
 '412x732' '320x568' '1440x900' '320x569' '424x753' '1024x600' '600x1024'
 '1280x1024' '1280x768' '1600x900' '540x960' '320x570' '480x800' '414x736'
 '1280x800' '601x962' '800x1280' '240x320' '1238x696' '640x360' '534x320'
 '2021x1137' '2560x1080' '800x480' '768x1024' '1829x1029' '360x592'
 '320x292' '320x339' '320x401' '480x854' '360x720' '320x480' '360x668'
 '1680x1050' '1920x1200' '720x1280' '486x863' '360x740' '1280x672'
 '393x786' '570x320' '240x274' '1821x1024' '1093x615' '432x719' '1152x864'
 '1280x752' '800x600' '282x356' '375x555' '406x722' '1152x720' '1256x640'
 '1684x947' '2286x1286' '2133x1200' '2000x1125' '962x601' '1440x810'
 '1438x808' '1024x820' '1301x731' '1344x840' '1088x615' '1607x904'
 '480x853' '2560x1440' '1600x1080' '403x671' '403x716' '1024x576'
 '1518x853' '820x615' '600x976' '320x427' '320x490' '1024x1024' '1336x751'


In [72]:
if debugging == True:
    print(input_train['operating_system_version'].unique()[0:100])

[nan 'Windows 10 ' 'Windows 7 ' 'Android 5.1.1' 'Android 7' 'Android 6'
 'Android 7.1.1' 'Android 6.0.1' 'Windows 8 ' 'iOS 11.3' 'Android 5.0.2'
 'Android 4.4.4' 'iOS 10.3.3' 'Linux ' 'Android 4.2.2' 'Windows 8.1 '
 'Windows XP ' 'Android 5.1' 'Android 4.1.2' 'iOS 11.2.2' 'iOS 11.2.6'
 'Mac OS X 10.11.6' 'Android 4.4.2' 'iOS 11.0.2' 'iOS 11.1.2'
 'Android 5.0.1' 'Android 8' 'Android 7.1.2' 'Android 4.3'
 'Windows Phone 10' 'iOS 5' 'Android 2.3.6' 'iOS 11.2.5' 'Mac OS X 10.13.4'
 'iOS 10.2.1' 'Windows Vista ' 'iOS 11.0.1' 'Mac OS X 10.11'
 'Mac OS X 10.12' 'Android 4.1.1' 'Windows Phone 8' 'iOS 9.3.5'
 'Android 8.1' 'Ubuntu ' 'Other ' 'Mac OS X 10.13.3' 'iOS 10.1.1'
 'iOS 11.4' 'Android 5' 'Android ' 'Android 4.0.3' 'FreeBSD '
 'Chrome OS 10452.85' 'Chrome OS 10323.67' 'Mac OS X 10.12.6'
 'Android 4.4.3' 'Mac OS X 10.10.5' 'Android 4.0.4' 'Windows Phone 8.1'
 'iOS 10.2' 'iOS 8.1.3' 'iOS 7.1.2' 'BlackBerry OS 10.3.2' 'iOS 5.0.1'
 'iOS 8.3' 'Android 3.2.2' 'iOS 11.0.3' 'iOS 11.1' 'Mac OS 

In [73]:
if debugging == True:
    print(input_train['browser_version'].unique()[0:100])

[nan 'Chrome 66.0' 'Chrome Mobile 43.0' 'Chrome Mobile 66.0' 'Chrome 67.0'
 'Chrome Mobile 39' 'Chrome 64.0' 'Chrome 65.0' 'Chrome 63.0'
 'Chrome Mobile 57.0' 'Chrome Mobile 65.0' 'Chrome Mobile 64.0'
 'Facebook 165' 'Firefox 58' 'Firefox 57' 'Firefox 59' 'IE 11'
 'Chrome 56.0' 'Mobile Safari 11' 'Chrome Mobile 34.0' 'Chrome Mobile 59.0'
 'Facebook 172' 'Facebook 173' 'Chrome Mobile 63.0' 'Chrome Mobile 55.0'
 'Mobile Safari 10.3' 'Firefox 60' 'Chrome Mobile 58.0' 'Chrome 55.0'
 'Edge 16.16299' 'Samsung Internet 3.5' 'Firefox 52' 'Chrome 51.0'
 'Chrome 49.0' 'Opera 52.0' 'Chrome Mobile 18.0' 'Android 4.1'
 'Chrome Mobile 62.0' 'Firefox 53' 'Facebook 90' 'Opera 53.0'
 'Chrome Mobile 56.0' 'Samsung Internet 6.4' 'Samsung Internet 6.2'
 'Chrome Mobile 50.0' 'Chrome 58.0' 'Android 4.3' 'Edge Mobile 14.14393'
 'Chrome 52.0' 'Chrome 41.0' 'Mobile Safari 5.1' 'Mobile Safari 10'
 'Chrome Mobile 45.0' 'Facebook 169' 'Facebook 170' 'Facebook 168'
 'Edge 17.17134' 'Android 2.3' 'Chrome Mobile 53.

In [74]:
if debugging == True:
    print(input_train.columns.values)

['timestamp' 'event' 'person' 'url' 'sku' 'model' 'condition' 'storage'
 'color' 'skus' 'search_term' 'staticpage' 'campaign_source'
 'search_engine' 'channel' 'new_vs_returning' 'city' 'region' 'country'
 'device_type' 'screen_resolution' 'operating_system_version'
 'browser_version']


> ## a) Features del timestamp

In [75]:
#input_train['day'] = pd.DataFrame(input_train['timestamp'].apply(lambda x: x.day))#, dummy_na=False) # Convierto a One Hot encoding, el último evita NaN
input_train['day'] = input_train['timestamp'].dt.day
input_train['weekDay']=input_train['timestamp'].dt.weekday
input_train['month'] =input_train['timestamp'].dt.month

In [76]:
#input_train['timestamp'].dt.dayofyear.min()

In [77]:
input_train['isweekEnd']=input_train['weekDay']<=4 # 0 es lunes, con esto verifico de lunes a viernes
input_train['isFortnite']=input_train['day']<=15 # Verifico quincena

In [78]:
grouped_input_train = input_train.groupby('person')

In [79]:
#Paso a segundos la interacción. Al final no lo usé, pero era la forma correcta parece ser
df_timeDelta = pd.DataFrame((grouped_input_train['timestamp'].max() - grouped_input_train['timestamp'].min()).rename('timeDelta').dt.total_seconds())
df_timeDelta.head()
new_vector_features = new_vector_features.join(df_timeDelta, on = 'person', how = 'right')

In [80]:
# Tiempo más cercano al timestamp al primero de junio
last_inter_to_june = pd.DataFrame((pd.to_datetime('06/01/2018') - grouped_input_train['timestamp'].max()).rename('last_inter_to_june').dt.total_seconds())
#last_inter_to_june.head()
new_vector_features = new_vector_features.join(last_inter_to_june, on = 'person', how = 'right')

In [81]:
# Tiempo más lejano al timestamp al primero de junio
first_inter_to_june = pd.DataFrame((pd.to_datetime('06/01/2018') - grouped_input_train['timestamp'].min()).rename('first_inter_to_june').dt.total_seconds())
#first_inter_to_june.head()
new_vector_features = new_vector_features.join(first_inter_to_june, on = 'person', how = 'right')

In [82]:
# Cantidad de interacciones en el tiempo especificado
new_vector_features['interactions'] = grouped_input_train['timestamp'].count()

In [83]:
new_vector_features.head() # Features hasta ahora

,label,resultKnown,timeDelta,last_inter_to_june,first_inter_to_june,interactions
person,,,,,,
0566e9c1,0,1,772618.0,26916.0,799534.0,68
6ec7ee77,0,1,0.0,336876.0,336876.0,2
abe7a2fb,0,1,9335392.0,174046.0,9509438.0,96
34728364,0,1,742549.0,388950.0,1131499.0,37
87ed62de,0,1,1409.0,1123199.0,1124608.0,17


In [84]:
#one_hot_weekday['isWeekend'] = one_hot_weekday[True]

In [85]:
#one_hot_weekday['isNotWeekend'] = one_hot_weekday[False]

In [86]:
#one_hot_weekday = one_hot_weekday.drop[True]


In [87]:
# Básico: Creo un histograma de eventos
count_weekday = pd.get_dummies(input_train['isweekEnd'], dummy_na=False) # Convierto a One Hot encoding, el último evita NaN
count_weekday['person'] = input_train['person'] # Agrego una columna
count_weekday = count_weekday.set_index('person')
#count_events.groupby('person').hist(bins = 3)
count_weekday = count_weekday.groupby('person') 

count_weekday = count_weekday.apply(lambda x: x.sum())

count_weekday['isNotWeekend'] = count_weekday[False]
count_weekday['isWeekend'] = count_weekday[True]
count_weekday = count_weekday.drop(columns = [False, True])

#count_weekday['weekDay' = count_weekday[False]
#Por las dudas, si quiero seleccionar la columna NaN, tengo que poner count_events[np.nan]

# Agregar al vector de features los que crea necesarios
new_vector_features = new_vector_features.join(count_weekday, on = 'person', how = 'left')

In [88]:
new_vector_features.head()

,label,resultKnown,timeDelta,last_inter_to_june,first_inter_to_june,interactions,isNotWeekend,isWeekend
person,,,,,,,,
0566e9c1,0,1,772618.0,26916.0,799534.0,68,5,63
6ec7ee77,0,1,0.0,336876.0,336876.0,2,0,2
abe7a2fb,0,1,9335392.0,174046.0,9509438.0,96,32,64
34728364,0,1,742549.0,388950.0,1131499.0,37,4,33
87ed62de,0,1,1409.0,1123199.0,1124608.0,17,1,16


In [89]:
# Básico: Creo un histograma de eventos
count_fortnite = pd.get_dummies(input_train['isFortnite'], dummy_na=False) # Convierto a One Hot encoding, el último evita NaN
count_fortnite['person'] = input_train['person'] # Agrego una columna
count_fortnite = count_fortnite.set_index('person')
#count_events.groupby('person').hist(bins = 3)
count_fortnite = count_fortnite.groupby('person') 

count_fortnite = count_fortnite.apply(lambda x: x.sum())

count_fortnite['isFortnite'] = count_fortnite[False]
count_fortnite['isNotFortnite'] = count_fortnite[True]
count_fortnite = count_fortnite.drop(columns = [False, True])

# Agregar al vector de features los que crea necesarios
new_vector_features = new_vector_features.join(count_fortnite, on = 'person', how = 'left')

In [90]:
# Básico: Creo un histograma de eventos
count_month = pd.get_dummies(input_train['month'], dummy_na=False) # Convierto a One Hot encoding, el último evita NaN
count_month['person'] = input_train['person'] # Agrego una columna
count_month = count_month.set_index('person')
#count_events.groupby('person').hist(bins = 3)
count_month = count_month.groupby('person') 

count_month = count_month.apply(lambda x: x.sum())

count_month = count_month.rename(columns={1: 'month1', 2: 'month2', 3: 'month3', 4: 'month4', 5:'month5'}) 

#count_weekday['isNotWeekend'] = count_weekday[False]
#count_weekday['isWeekend'] = count_weekday[True]
#count_weekday = count_weekday.drop(columns = [False, True])

#count_weekday['weekDay' = count_weekday[False]
#Por las dudas, si quiero seleccionar la columna NaN, tengo que poner count_events[np.nan]

# Agregar al vector de features los que crea necesarios
new_vector_features = new_vector_features.join(count_month, on = 'person', how = 'left')

In [91]:
new_vector_features.head()

,label,resultKnown,timeDelta,last_inter_to_june,first_inter_to_june,interactions,isNotWeekend,isWeekend,isFortnite,isNotFortnite,month1,month2,month3,month4,month5
person,,,,,,,,,,,,,,,
0566e9c1,0,1,772618.0,26916.0,799534.0,68,5,63,68,0,0,0,0,0,68
6ec7ee77,0,1,0.0,336876.0,336876.0,2,0,2,2,0,0,0,0,0,2
abe7a2fb,0,1,9335392.0,174046.0,9509438.0,96,32,64,32,64,0,12,16,38,30
34728364,0,1,742549.0,388950.0,1131499.0,37,4,33,37,0,0,0,0,0,37
87ed62de,0,1,1409.0,1123199.0,1124608.0,17,1,16,17,0,0,0,0,0,17


In [92]:
# Ver si le agrego info del día, y tiempo al próximo feriado, al último o cosas así
# Ideas: Por mes? Por día? Cantidad de eventos en fines de semana? Cantidad de eventos en días de semana?
# Cuenta de algún evento particular?
# Histograma de eventos de días de semana vs fines de semana?
# media y varianza de acuerdo al día del mes?
# Algo relacionado a la hora?

> ## b) Features de events

In [93]:
# Diferentes eventos
input_train['event'].unique()

array(['checkout', 'visited site', 'generic listing', 'ad campaign hit',
       'viewed product', 'brand listing', 'search engine hit',
       'conversion', 'searched products', 'staticpage', 'lead'], dtype=object)

In [94]:
# Básico: Creo un histograma de eventos
count_events = pd.get_dummies(input_train['event'], dummy_na=False) # Convierto a One Hot encoding, el último evita NaN
count_events['person'] = input_train['person'] # Agrego una columna
count_events = count_events.set_index('person')
#count_events.groupby('person').hist(bins = 3)
count_events = count_events.groupby('person') 

count_events = count_events.apply(lambda x: x.sum())
#Por las dudas, si quiero seleccionar la columna NaN, tengo que poner count_events[np.nan]

# Agregar al vector de features los que crea necesarios
new_vector_features = new_vector_features.join(count_events, on = 'person', how = 'left')

In [95]:
count_events.head()

,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,0,0,3,0,1,0,0,0,0,0,2
00091926,15,25,2,0,0,0,0,0,0,372,34
00091a7a,1,5,0,0,0,0,0,0,0,3,1
000ba417,1,24,6,1,14,0,1,0,0,153,6
000c79fe,1,0,1,0,1,0,1,9,0,3,1


In [96]:
addTimeToEvent = True

In [97]:
if addTimeToEvent == True:
    ev = input_train[['person', 'event', 'timestamp']]

    different_events = ev['event'].unique()
    for i in different_events:
        #print (i)
        b = 'time_to_' + i
        print(b)

    #eev = ev.groupby('person')
    last_date = (pd.to_datetime('06/01/2018'))# - grouped_input_train['timestamp'].max()).rename('last_inter_to_june').dt.total_seconds())

    ev = input_train[['person', 'event', 'timestamp']]
    ev['timestamp'] = ev['timestamp']
    ev = (last_date - ev.groupby(['person', 'event'])['timestamp'].max()).dt.total_seconds().to_frame(name='dist').unstack().reset_index()
    ev.columns = ['person'] + [ 'june_to_last_' + x[1] for x in ev.columns][1:]
    ev = ev.fillna(0).set_index('person')

    new_vector_features = new_vector_features.join(ev, on = 'person', how = 'left')

time_to_checkout
time_to_visited site
time_to_generic listing
time_to_ad campaign hit
time_to_viewed product
time_to_brand listing
time_to_search engine hit
time_to_conversion
time_to_searched products
time_to_staticpage
time_to_lead


/home/diego/.local/anaconda3/envs/tfdeeplearning/lib/python3.5/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [98]:
# Recordar agregar url, cuando aparece es porque tiene ad_campain_hit
# Algo relacionado a profundidad del hit?
# Algo relacionado a la varianza del hit?

> ## c) Features de Condition

In [99]:
input_train['condition'].unique() # Sem touch ID es sin lector de huellas, normalmente los IPhone|

array(['Muito Bom', nan, 'Bom', 'Excelente', 'Novo', 'Bom - Sem Touch ID'], dtype=object)

In [100]:
# Función que retorna un valor de acuerdo a la calidad del celular:
# nan, 'Muito Bom', 'Bom', 'Excelente', 'Bom - Sem Touch ID', 'Novo'
def getCondition(str):
    if not str:
        return 0
    elif str == 'Bom - Sem Touch ID': # Ojo que en la lista del TP1 no aparecen pero acá si
        return 1
    elif str == 'Bom':
        return 2
    elif str == 'Muito Bom':
        return 3
    elif str == 'Excelente':
        return 4
    elif str == 'Novo':
        return 5
    else:
        assert('Entrada desconocida')

getCondition('Bom') # Test

2

In [101]:
# Ojo con los que no son condition...
cond_temp = input_train['condition'].apply(lambda x: getCondition(x))
one_hot_condition = pd.get_dummies(cond_temp, dummy_na=False) # Si pongo true me genera una columna más con los nan
one_hot_condition['person'] = input_train['person'] # Agrego una columna
one_hot_condition = one_hot_condition.set_index('person')
one_hot_condition = one_hot_condition.groupby('person') 
one_hot_condition = one_hot_condition.apply(lambda x: x.sum())
new_vector_features = new_vector_features.join(one_hot_condition, on = 'person', how = 'left')

# Normalizar?
# Varianza y valor medio?

In [102]:
#aa = pd.DataFrame([[2,3],[2,np.nan],[3,3]])
#aa[1].mean() #Para ver que pasa si hay un nan, está bien, no lo tiene en cuenta

In [103]:
del cond_temp
del one_hot_condition

In [104]:
cond_temp = input_train['condition'].apply(lambda x: getCondition(x))
cond_temp_mean = pd.DataFrame(cond_temp.rename('cond_mean'))
cond_temp_mean['person'] = input_train['person']
cond_temp_mean = cond_temp_mean.set_index('person')
cond_temp_mean = cond_temp_mean.groupby('person')
cond_temp_mean = cond_temp_mean.apply(lambda x:x.mean())
new_vector_features = new_vector_features.join(cond_temp_mean, on = 'person', how = 'left')
new_vector_features['cond_mean'].fillna(6, inplace=True) # Si son todos nan pone un 6

In [105]:
del cond_temp
del cond_temp_mean

In [106]:
cond_temp = input_train['condition'].apply(lambda x: getCondition(x))
cond_temp_var = pd.DataFrame(cond_temp.rename('cond_var'))
cond_temp_var['person'] = input_train['person']
cond_temp_var = cond_temp_var.set_index('person')
cond_temp_var = cond_temp_var.groupby('person')
cond_temp_var = cond_temp_var.apply(lambda x:x.var())
new_vector_features = new_vector_features.join(cond_temp_var, on = 'person', how = 'left')
new_vector_features['cond_var'].fillna(6, inplace=True) # Si son todos nan pone un 6

In [107]:
del cond_temp
del cond_temp_var

> ## Feature de ...

In [108]:
# Ver que hacer con model: marca, modelo, rango de precios estimado?
# Marca del sistema operativo
def getBrand(str):
    return str.partition(' ')[0]
getBrand('Asus zenpad')

'Asus'

## Feature de Capacidad 

In [110]:
# Storage
# TODO: Ver si conviene ir logarítmicamente, o cómo. Que hago con el 512? Hay que probar de las dos formas
# La otra es asignarle un número entero
# No debería tener un 0?
def getCapacity(str):
    my_list = [ '32GB', '64GB', '128GB', '256GB', '8GB', '16GB', '4GB', '512MB']
    to_replace = [  32,     64,     128,     256,     8,     16,     4,     0.512 ]
    if str in my_list:
        return to_replace[my_list.index(str)]
    else:
        return 0
getCapacity('32GB') # Test
def getCapacityCat(str):
    my_list = [ '32GB', '64GB', '128GB', '256GB', '8GB', '16GB', '4GB', '512MB']
    to_replace = [  5,     6,     7,     8,     3,     4,     2,     1 ]
    if str in my_list:
        return to_replace[my_list.index(str)]
    else:
        return 0
getCapacity('32GB') # Test
getCapacityCat('32GB') # Test

5

In [111]:
addStorage = True

In [145]:
knownLabels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19414 entries, 0566e9c1 to 824fa9e3
Data columns (total 2 columns):
label          19414 non-null int64
resultKnown    19414 non-null int64
dtypes: int64(2)
memory usage: 455.0+ KB


In [112]:
if addStorage == True:
    # Ojo con los que no son condition...
    storage_temp = input_train['storage'].apply(lambda x: getCapacity(x))
    storage_count = pd.get_dummies(storage_temp, dummy_na=False, prefix='storage_count') # Si pongo true me genera una columna más con los nan
    storage_count['person'] = input_train['person'] # Agrego una columna
    storage_count = storage_count.set_index('person')
    storage_count = storage_count.groupby('person') 
    storage_count = storage_count.apply(lambda x: x.sum())
    new_vector_features = new_vector_features.join(storage_count, on = 'person', how = 'left').drop( 'storage_count_0.0', axis = 1) # El 0 es interacciones sin storage

    # Normalizar?
    # Varianza y valor medio?

In [113]:
if addStorage == True:
    storage_temp = input_train['storage'].apply(lambda x: getCapacityCat(x))
    storage_temp_mean = pd.DataFrame(storage_temp.rename('storage_mean'))
    storage_temp_mean['person'] = input_train['person']
    storage_temp_mean = storage_temp_mean.set_index('person')
    storage_temp_mean = storage_temp_mean.groupby('person')
    storage_temp_mean = storage_temp_mean.apply(lambda x:x.mean())
    new_vector_features = new_vector_features.join(storage_temp_mean, on = 'person', how = 'left')
    new_vector_features['storage_mean'].fillna(0, inplace=True) # Si son todos nan pone un 6

In [114]:
if addStorage == True:
    storage_temp = input_train['storage'].apply(lambda x: getCapacityCat(x))
    storage_temp_var = pd.DataFrame(storage_temp.rename('storage_var'))
    storage_temp_var['person'] = input_train['person']
    storage_temp_var = storage_temp_var.set_index('person')
    storage_temp_var = storage_temp_var.groupby('person')
    storage_temp_var = storage_temp_var.apply(lambda x:x.var())
    new_vector_features = new_vector_features.join(storage_temp_var, on = 'person', how = 'left')
    new_vector_features['storage_var'].fillna(0, inplace=True) # Si son todos nan pone un 6

In [115]:
# Qué hacer con color?

In [116]:
# Qué hacer con respecto a la campaña:
# Qué hacer con search_term, staticpage, campaign_source, search_engine
# Channel

## Feature de Channel 

In [117]:
print(input_train['channel'].unique())

[nan 'Referral' 'Direct' 'Paid' 'Organic' 'Social' 'Email' 'Unknown']


In [120]:
# Básico: Creo un histograma de eventos
count_channel = pd.get_dummies(input_train['channel'], dummy_na=False, prefix = 'channel') # Convierto a One Hot encoding, el último evita NaN
count_channel['person'] = input_train['person'] # Agrego una columna
count_channel = count_channel.set_index('person')
#count_events.groupby('person').hist(bins = 3)
count_channel = count_channel.groupby('person') 

count_channel = count_channel.apply(lambda x: x.sum())
#Por las dudas, si quiero seleccionar la columna NaN, tengo que poner count_events[np.nan]

# Agregar al vector de features los que crea necesarios
new_vector_features = new_vector_features.join(count_channel, on = 'person', how = 'left')

In [121]:
# New vs Returning
# No hay un New por cada persona, pero anda por ahí (19414 vs 19126) y puede haber varios returning
#input_train[input_train['new_vs_returning'] =='Returning']['person'].value_counts()
#input_train[input_train['event'] =='visited site']['person'].value_counts()

In [122]:
# Relativo a la ubicación: 
#   Qué hacer con city, region, country

In [123]:
# Relativo al sistema operativo:
#   Qué hacer con device_type, screen_resolution, operative_system_version, browser_version

## Feature de device type

In [124]:
addDeviceType = True

In [125]:
if addDeviceType == True:
    # Básico: Creo un histograma de eventos
    count_device_type = pd.get_dummies(input_train['device_type'], dummy_na=False, prefix='device_type') # Convierto a One Hot encoding, el último evita NaN
    count_device_type['person'] = input_train['person'] # Agrego una columna
    count_device_type = count_device_type.set_index('person')
    #one_hot_events.groupby('person').hist(bins = 3)
    count_device_type = count_device_type.groupby('person') 

    count_device_type = count_device_type.apply(lambda x: x.sum())

    #count_device_type['count_device_type'] = one_hot_weekday[False]
    #count_device_type['count_device_type'] = one_hot_weekday[True]
    #count_device_type = count_device_type.drop(columns = [False, True])

    #one_hot_weekday['weekDay' = one_hot_weekday[False]
    #Por las dudas, si quiero seleccionar la columna NaN, tengo que poner one_hot_events[np.nan]


    # Agregar al vector de features los que crea necesarios
    new_vector_features = new_vector_features.join(count_device_type, on = 'person', how = 'left')

In [127]:
if addDeviceType == True:
    # Agrego los valores anteriores porcentualmente
    count_device_type_total = count_device_type.sum(axis = 1)
    count_device_type_perc = count_device_type.copy()[[]] # Creo dataframe vacío con los mismos índices
    count_device_type_perc['perc_computer'] = pd.DataFrame(count_device_type.device_type_Computer/count_device_type_total)
    count_device_type_perc['perc_tablet'] = count_device_type.device_type_Tablet/count_device_type_total
    count_device_type_perc['perc_smartphone'] = count_device_type.device_type_Smartphone/count_device_type_total
    count_device_type_perc['perc_unknown'] = count_device_type.device_type_Unknown/count_device_type_total

    new_vector_features = new_vector_features.join(count_device_type_perc, on = 'person', how = 'left')
    new_vector_features = new_vector_features.fillna(0)

## Feature 

In [128]:
#count_device_type[(count_device_type.Computer>0) * (count_device_type.Smartphone>0)]

In [129]:
#count_device_type_total[(count_device_type.Computer>0) * (count_device_type.Smartphone>0)].head()

In [130]:
new_vector_features.head()

,label,resultKnown,timeDelta,last_inter_to_june,first_inter_to_june,interactions,isNotWeekend,isWeekend,isFortnite,isNotFortnite,...,channel_Social,channel_Unknown,device_type_Computer,device_type_Smartphone,device_type_Tablet,device_type_Unknown,perc_computer,perc_tablet,perc_smartphone,perc_unknown
person,,,,,,,,,,,,,,,,,,,,,
0566e9c1,0,1,772618.0,26916.0,799534.0,68,5,63,68,0,...,0,0,0,17,0,0,0.0,0.0,1.0,0.0
6ec7ee77,0,1,0.0,336876.0,336876.0,2,0,2,2,0,...,0,0,0,1,0,0,0.0,0.0,1.0,0.0
abe7a2fb,0,1,9335392.0,174046.0,9509438.0,96,32,64,32,64,...,2,0,0,22,0,0,0.0,0.0,1.0,0.0
34728364,0,1,742549.0,388950.0,1131499.0,37,4,33,37,0,...,0,0,0,4,0,0,0.0,0.0,1.0,0.0
87ed62de,0,1,1409.0,1123199.0,1124608.0,17,1,16,17,0,...,0,0,1,0,0,0,1.0,0.0,0.0,0.0


In [131]:
input_train.iloc[:,0:2].head()

,timestamp,event
1507286,2018-05-17 12:27:47,checkout
2336761,2018-05-17 16:21:54,visited site
2122051,2018-05-17 16:22:06,generic listing
1505383,2018-05-17 16:28:37,checkout
2336760,2018-05-17 13:44:59,visited site


In [132]:
input_train[['event', 'url', 'timestamp']][~input_train.url.isnull()].head(10)

,event,url,timestamp
227610,ad campaign hit,/comprar/iphone/6s,2018-05-23 01:09:18
100351,ad campaign hit,/comprar/samsung/galaxy-s6-edge-plus,2018-05-08 23:30:44
172395,ad campaign hit,/comprar/iphone,2018-05-31 19:51:49
62056,ad campaign hit,/comprar/iphone,2018-05-29 01:58:14
208388,ad campaign hit,/comprar/iphone/7-plus,2018-05-20 01:10:39
182822,ad campaign hit,/comprar/iphone/6-plus,2018-05-23 01:09:25
208774,ad campaign hit,/comprar/iphone/iphone-6s-plus,2018-05-20 01:01:27
114267,ad campaign hit,/comprar/samsung/galaxy-s7-edge,2018-05-09 00:25:05
210999,ad campaign hit,/comprar/samsung/galaxy-s6-edge-plus,2018-05-22 00:55:28
67536,ad campaign hit,/comprar/samsung/galaxy-s6-flat,2018-05-11 02:23:38


In [133]:
input_train['event'][~input_train.url.isnull()].unique()

array(['ad campaign hit'], dtype=object)

In [134]:
input_train['url'][~input_train.url.isnull()].unique()

array(['/comprar/iphone/6s', '/comprar/samsung/galaxy-s6-edge-plus',
       '/comprar/iphone', '/comprar/iphone/7-plus',
       '/comprar/iphone/6-plus', '/comprar/iphone/iphone-6s-plus',
       '/comprar/samsung/galaxy-s7-edge',
       '/comprar/samsung/galaxy-s6-flat', '/comprar/samsung',
       '/comprar/samsung/galaxy-s8-plus', '/comprar/samsung/galaxy-s8',
       '/comprar/iphone/', '/comprar/samsung/a3-duos', '/',
       '/comprar/samsung/galaxy-s5', '/comprar/lg/g3-d855',
       '/comprar/iphone/iphone-se',
       '/comprar/samsung/samsung-gran-prime-duos-tv',
       '/comprar/samsung/galaxy-s7', '/comprar/samsung/galaxy-win-duos',
       '/comprar/motorola/moto-g-2a-geracao', '/comprar/iphone/iphone-5c',
       '/comprar/samsung/galaxy-j1-mini',
       '/comprar/samsung/galaxy-s4-mini-duos', '/comprar/motorola',
       '/comprar/samsung/galaxy-s5-duos', '/comprar/motorola/',
       '/comprar/samsung/galaxy-j7-prime', '/comprar/iphone/6',
       '/comprar/iphone/iphone-5s',
    

# https://blog.datadive.net/selecting-good-features-part-iii-random-forests/

# Uso de varias técnicas: https://www.kaggle.com/niklasdonges/end-to-end-project-with-python

# Lo usa mal! OJO

In [135]:
#input_train[input_train['new_vs_returning'] =='Returning']
input_train[input_train['event'] == 'checkout'][['condition','event']]

,condition,event
1507286,Muito Bom,checkout
1505383,Bom,checkout
1507716,Bom,checkout
208585,Excelente,checkout
213543,Muito Bom,checkout
244902,Bom,checkout
912538,Muito Bom,checkout
912515,Muito Bom,checkout
848525,Bom,checkout
924550,Bom,checkout


In [137]:
new_vector_features_to_predict = new_vector_features.drop('resultKnown', axis = 1)[new_vector_features['resultKnown']==0] # Datos para predecir
new_vector_features_to_predict.shape

(19415, 68)

In [138]:
new_vector_features_known = new_vector_features.drop('resultKnown', axis = 1)[new_vector_features['resultKnown']==1] # Datos para predecir
new_vector_features_known.shape

(19414, 68)

In [139]:
to_submit = pd.read_csv(_PATH_DATASET + 'trocafone_kaggle_submit_sample_all_0.csv')
new_vector_features_to_predict = new_vector_features_to_predict.reindex(to_submit.person)

In [140]:
new_vector_features_known.to_csv(_PATH_DATASET + 'out_features_processed_train.csv', sep=',', encoding='utf-8')
new_vector_features_to_predict.to_csv(_PATH_DATASET + 'out_features_processed_to_predict.csv', sep=',', encoding='utf-8')

In [141]:
new_vector_features.head()

,label,resultKnown,timeDelta,last_inter_to_june,first_inter_to_june,interactions,isNotWeekend,isWeekend,isFortnite,isNotFortnite,...,channel_Social,channel_Unknown,device_type_Computer,device_type_Smartphone,device_type_Tablet,device_type_Unknown,perc_computer,perc_tablet,perc_smartphone,perc_unknown
person,,,,,,,,,,,,,,,,,,,,,
0566e9c1,0,1,772618.0,26916.0,799534.0,68,5,63,68,0,...,0,0,0,17,0,0,0.0,0.0,1.0,0.0
6ec7ee77,0,1,0.0,336876.0,336876.0,2,0,2,2,0,...,0,0,0,1,0,0,0.0,0.0,1.0,0.0
abe7a2fb,0,1,9335392.0,174046.0,9509438.0,96,32,64,32,64,...,2,0,0,22,0,0,0.0,0.0,1.0,0.0
34728364,0,1,742549.0,388950.0,1131499.0,37,4,33,37,0,...,0,0,0,4,0,0,0.0,0.0,1.0,0.0
87ed62de,0,1,1409.0,1123199.0,1124608.0,17,1,16,17,0,...,0,0,1,0,0,0,1.0,0.0,0.0,0.0


In [133]:
#X_train = new_vector_features.drop("label", axis=1)
#Y_train = new_vector_features["label"]
#X_test  = test_df.drop("PassengerId", axis=1).copy()
dasdfs desde acá me fuí a la otra notebook, fuerzo error. sirve para tener referencias

SyntaxError: invalid syntax (<ipython-input-133-26245e3454a6>, line 4)

In [ ]:
new_vector_features.reset_index()[new_vector_features.reset_index()['person']=='0566e9c1']

In [ ]:
new_vector_features.head()

In [ ]:
# Algorithms
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB

In [ ]:
'''
This estimator implements regularized linear models with stochastic
gradient descent (SGD) learning: the gradient of the loss is estimated
each sample at a time and the model is updated along the way with a
decreasing strength schedule (aka learning rate). SGD allows minibatch
(online/out-of-core) learning, see the partial_fit method.
For best results using the default learning rate schedule, the data should
have zero mean and unit variance.
'''
sgd = linear_model.SGDClassifier(max_iter=5, tol=None)

In [ ]:
'''
Ajustar el clasificador
Fit linear model with Stochastic Gradient Descent.
'''
sgd.fit(X_train, y_train)

In [ ]:
'''
Predict class labels for samples in X.
'''
Y_pred = sgd.predict(X_test)
sgd.score(X_train, y_train)



In [ ]:
'''
Returns the mean accuracy on the given test data and labels.
'''
acc_sgd = round(sgd.score(X_test, y_test) * 100, 2)

acc_sgdPred = round(sgd.score(X_test, Y_pred) * 100, 2)

print(round(acc_sgd,2,), "%")
print(round(acc_sgdPred,2,), "%")

In [ ]:
# Random Forest
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)

Y_prediction = random_forest.predict(X_test)

random_forest.score(X_test, y_test)
acc_random_forest = round(random_forest.score(X_test, y_test) * 100, 2)
print(round(acc_random_forest,2,), "%")

In [ ]:
# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

Y_pred = logreg.predict(X_test)

acc_log = round(logreg.score(X_test, y_test) * 100, 2)
print(round(acc_log,2,), "%")



In [ ]:
# KNN
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)

Y_pred = knn.predict(X_test)

acc_knn = round(knn.score(X_train, y_train) * 100, 2)
print(round(acc_knn,2,), "%")

In [ ]:
# Gaussian Naive Bayes
gaussian = GaussianNB()
gaussian.fit(X_train, y_train)

Y_pred = gaussian.predict(X_test)

acc_gaussian = round(gaussian.score(X_train, y_train) * 100, 2)
print(round(acc_gaussian,2,), "%")

In [ ]:


# Perceptron
perceptron = Perceptron(max_iter=5)
perceptron.fit(X_train, y_train)

Y_pred = perceptron.predict(X_test)

acc_perceptron = round(perceptron.score(X_train, y_train) * 100, 2)
print(round(acc_perceptron,2,), "%")



In [ ]:
# Linear SVC
linear_svc = LinearSVC()
linear_svc.fit(X_train, y_train)

Y_pred = linear_svc.predict(X_test)

acc_linear_svc = round(linear_svc.score(X_train, y_train) * 100, 2)
print(round(acc_linear_svc,2,), "%")

In [ ]:


# Decision Tree
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)

Y_pred = decision_tree.predict(X_test)

acc_decision_tree = round(decision_tree.score(X_train, y_train) * 100, 2)
print(round(acc_decision_tree,2,), "%")



In [ ]:
results = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 
              'Decision Tree'],
    'Score': [acc_linear_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_decision_tree]})
result_df = results.sort_values(by='Score', ascending=False)
result_df = result_df.set_index('Score')
result_df.head(9)

In [ ]:
y_test.values

In [ ]:
sum(abs(Y_pred - y_test))

In [ ]:
Y_pred.shape

In [ ]:
296/5835

In [ ]:


from sklearn.model_selection import cross_val_score
rf = RandomForestClassifier(n_estimators=100)
scores = cross_val_score(rf, X_train, y_train, cv=10, scoring = "accuracy")



In [ ]:
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

# Feature importance

In [ ]:
importances = pd.DataFrame({'feature':X_train.columns,'importance':np.round(random_forest.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')

In [ ]:
importances.head(20)

In [ ]:
importances.plot.bar()

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
predictions = cross_val_predict(random_forest, X_train, y_train, cv=3)
confusion_matrix(y_train, predictions)

In [ ]:
from sklearn.metrics import precision_recall_curve

# getting the probabilities of our predictions
y_scores = random_forest.predict_proba(X_test)
y_scores = y_scores[:,1]

precision, recall, threshold = precision_recall_curve(y_test, y_scores)

In [ ]:
import matplotlib.pyplot as plt
def plot_precision_and_recall(precision, recall, threshold):
    plt.plot(threshold, precision[:-1], "r-", label="precision", linewidth=5)
    plt.plot(threshold, recall[:-1], "b", label="recall", linewidth=5)
    plt.xlabel("threshold", fontsize=19)
    plt.legend(loc="upper right", fontsize=19)
    plt.ylim([0, 1])

plt.figure(figsize=(14, 7))
plot_precision_and_recall(precision, recall, threshold)
plt.show()

In [ ]:
knownLabels.head()